In [3]:
import time
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from functools import reduce
from itertools import chain
from scipy.stats import stats
from scipy.stats import rankdata
from scipy.optimize import minimize

from mpl_toolkits.mplot3d import Axes3D

from pyspark.mllib.linalg.distributed import IndexedRowMatrix, IndexedRow
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import create_map, col, to_date, date_format, year, month, dayofmonth, when, lit, lag, array, explode, struct, udf, first
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg, count, stddev as spark_stddev
from pyspark.sql.types import FloatType, StructField, StructType, DateType, IntegerType, ArrayType
from pyspark.sql import SparkSession, Window, DataFrame
from pylab import *
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [4]:
pd.set_option('display.max_columns', 10000000)
pd.set_option('display.max_rows', 10000000)
pd.set_option('display.width', 10000000)

------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Defining Spark Session for pseudo-distributed computing:

In [5]:
spark = SparkSession.builder.appName('Portfolio_Optimization').getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=Portfolio_Optimization>

# Reading persisted Portfolio Yields dataframe:

In [6]:
portfolio_yield_window_path = '/data/core/fince/data/portfolioOptimization/portfolio_yield_window/'
portfolio_yield_df = spark.read.parquet(portfolio_yield_window_path)

In [7]:
portfolio_yield_df.limit(10).toPandas()

SCOTIAG    AXESCP   BMERGOB  BMRGOB25   VALUEF4   BLKDIA7   BLKGUB1   GBMGUBL   INVEXGU  NTEGUB13  NTEGUB15   PRINFGU   ST&ER1X   STERGOB   SURCETE   VECTPRE    FONDEO   SCOTIA1  BMERLIQ28   DINBUR1  FONSER130     GBMF2   INBUMAX   INVEXCP   ACTIREN     SUR1E   MONEX28  BLKDIN45  VECTRF48  NTEMP+50   SBANKCP  BMERLIQ52   BMERTES  BMRGOB254  BLKDIA55  BLKLIQ/A  BLKPLUSA  FONSER161     PYMES   ST&ER1P   STER10P     SUPER   INVEXDX  HSBCEMP69  VECTRF71   SCOTIA2  ACTIGOB74   AFIRPLU  AXESMP76  BLK1LIQA  HSBCMP83  INVEXMP85  NTEMP+86  PRINFMP88  PRINMAS89  VECTMD92  HSBCEMP93   ACTIMED   ALTERNA     GBMF3     GBMM3   HSBCCOR     NTEDP  NTEMP+105   SURCORP  FINDE1110       SBANKMP  INVEXMP117    DMEDIO  PRINFMP119  NTEMP+120   BONDDIA  PRINMAS125  AXESMP130    GBMGUB   SCOTILP   INBUREX   PRINGLP    SUR30E   SURBONO    AXESLP   SCOTLPG   SCOTIMB   NAFINTR   PRINFTR   STEREAL    SURUDI   FT/REAL   VALUEF2   VALUEF7   SCOTUDI   SCOTDOL  BBVADOL183  INVEXCO185  VECTCOB188  HSBCDOL189   SBANKDL  BBVADOL195  INVEXCO197  VECTCOB200  HSBCDOL201   SCOT-FX   ACTICOB   BLKDOLS    PRGLOB   ST&ERUS    SURUSD   TEMGBIA   NAVIGTR   SCOTIPC   ACTIVAR       BMERIND   NAFINDX    SURIPC   VECTIND   ACTIPAT    BLKPAT    BLKIPC    GBMCRE    GBMMOD   HSBCBOL   INVEXMX      MAYA    NTESEL   PRINRVA    SURPAT    VECTPA   ACTINMO   BLKFIBR   SCOTUSA  ACTI500261  VECTSIC262  BLKUSEQ265  FRANOPR266  FRANUSA267  NTEUSA269  STERDOW270  ACTI500272  BLKUSEQ276  FRANOPR277  FRANUSA278  NTEUSA280  STERDOW281  VECTSIC282   AXESGLO   BLKINT1    GBMTRV   INVEXTK    NTEGLA   SURGLOB   SCOTEUR   AXESEDM   BBVAE50   SCOTI12   GBMPCON   PRINLS1        SURMOD  SVIVE50310   SCOTI14   ELITE/C  GBMPMOD314    HSBCF2  HSBCF3317  PRINLS2319  SUR2026320  SURCRE321  SVIVE35322   SBANK50  DIVER/A324  ELITE/M326  GOLD4MA329  PRINLS3331  SUR2034334  SUR2042335  SURAGR337  SVIVE20338   SCOTQNT  DIVER/A340  ELITE/M342  GBMPMOD344  GOLD4MA345   GOLD5MA  HSBCF3347  SURCRE352  SURAGR353    HSBCF4  PRINLS2357  PRINLS3358   GBMPICT   SVIVE60   SUR2018   HSBCJUB  SUR2026370   HSBC-50  SVIVE50376  SUR2034377   HSBC-60  SVIVE35383  SUR2042384   HSBC-70  SVIVE20390   HSBC-80  FINDE1400  ACTIGOB401  AXESMP402  HSBCMP405  INVEXMP407   NAFFP28  PRINFMP410  PRINMAS411  VECTMD414 operation_date
0 -0.000404 -0.000149 -0.000422 -0.000405 -0.000347 -0.000412 -0.000407 -0.000404 -0.000348 -0.000200 -0.000401 -0.000385 -0.000267 -0.000384 -0.000341 -0.000371 -0.000202 -0.000217  -0.000211 -0.000153  -0.000174 -0.000214 -0.000177 -0.000127 -0.000246 -0.000199 -0.000207 -0.000203 -0.000204 -0.000203 -0.000223  -0.000211 -0.000206  -0.000222 -0.000206 -0.000173 -0.000206  -0.000174 -0.000154 -0.000163 -0.000155 -0.000175 -0.000148  -0.000163 -0.000204 -0.000138  -0.000203 -0.000213 -0.000154 -0.000199 -0.000168  -0.000172 -0.000203  -0.000132  -0.000056 -0.000170  -0.000163 -0.000238 -0.000253 -0.000116 -0.000170 -0.000041 -0.000217  -0.000203 -0.000076  -0.000140 -1.336693e-04   -0.000172 -0.000280   -0.000132  -0.000203 -0.000182   -0.000056  -0.000154 -0.000195  0.000092  0.000217  0.000277  0.000372  0.000768  0.000012  0.000006  0.000265  0.000263  0.000756  0.000945  0.000665  0.000712  0.001083  0.000896  0.001047  0.003809    0.003866    0.003202    0.003888    0.003091  0.003873    0.003866    0.003202    0.003888    0.003091  0.001573  0.003924  0.002550  0.001245  0.002916  0.002553  0.004039  0.000744 -0.007192 -0.007019 -5.284610e-03 -0.007037 -0.007290 -0.007076 -0.005584 -0.007299 -0.008382 -0.002073 -0.000449 -0.007122 -0.005154 -0.005205 -0.008272 -0.007304 -0.009261 -0.009140 -0.011282 -0.019168 -0.011258   -0.009090   -0.007613    0.007151   -0.010189   -0.004312  -0.008963   -0.008517   -0.009090    0.007151   -0.010189   -0.004312  -0.008963   -0.008517   -0.007613 -0.005546  0.007526 -0.007872 -0.007816 -0.003895  0.023515  0.002702  0.002850  0.009655 -0.001490 -0.000423 -0.000627 -3.052937e-03   -0.006211 -0.002024  0.000001   -0.000742 -0.001582  -0.002339   -0.003492    0.000087 

# From now on we construct yield portfolio matrix for Risk Parity process.

In [9]:
field_array = portfolio_yield_df.columns[:-1]
print('test with', len(field_array), 'funds\n')

monthly_return = np.array(portfolio_yield_df.select(*field_array).collect())
print("numpy matrix generated correctly!\n")
print('monthly_return matrix:\n', monthly_return)

test with 211 funds

numpy matrix generated correctly!

monthly_return matrix:
 [[ -4.03531517e-04  -1.49306118e-04  -4.21638278e-04 ...,  -1.31695998e-04
   -5.64483602e-05  -1.69570039e-04]
 [ -1.08301688e-04  -4.31589093e-05  -1.14076923e-04 ...,  -1.28230898e-04
   -2.08582693e-04  -1.09002844e-04]
 [ -1.03740924e-04  -6.38387403e-05  -1.03136047e-04 ...,  -9.41218795e-05
    2.64438926e-05  -5.25563589e-05]
 ..., 
 [ -1.26953831e-04  -3.56847525e-05  -1.19272283e-04 ...,  -5.63834996e-04
   -1.50472534e-03  -2.12792538e-04]
 [ -1.72035421e-04  -1.27096848e-04  -1.96661294e-04 ...,  -2.23718400e-04
   -2.56411067e-04  -2.03920502e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


### Get model parameters Class:
#### This calss will have defined functions that will help us as utilities for creating specific matrices objects.

In [567]:
class ModelParameters(object):
    
    def get_shape_matrix(numpy_matrix):
        """
        Gets numpy.array matrix shape.
        :param numpy_matrix: yield numpy.array matrix.
        :return: tuple with matrix shape (rows, cols).
        """
        if isinstance(numpy_matrix, (np.ndarray)):
            n_rows, n_cols = numpy_matrix.shape[0], numpy_matrix.shape[1]
        else:
            print("matrix not valid, must be numpy.array object with float terms.")
            return None
        
        return n_rows, n_cols
    
    def set_time_prediction(numpy_matrix, time_predict=None, n_month=None):
        """
        Set time prediction function it creates an returns time prediction window, 
        starting month variable for analysis and historical observations variable.
        :param numpy_matrix: yield portfolio numpy.array matrix.
        :param time_predict: time to predict int, 11 default.
        :param n_month: number of months int, 1 default.
        :return: tuple with initial model optimization params.
        """
        if not isinstance(numpy_matrix, (np.ndarray)):
            print("matrix not valid, must be numpy.array object with float terms.")
            return None
        else:
            n_rows, n_cols = ModelParameters.get_shape_matrix(numpy_matrix)
            
        if time_predict is None:
            time_predict = 11
        if n_month is None:
            n_month = 1

        if time_predict is not None:
            if isinstance(time_predict, int):
                time_predict = time_predict
            else:
                print("time predict params not valid, must be integer.")
                return None
        
        if n_month is not None:
            if isinstance(n_month, int):
                n_month =  n_month
            else:
                print("number of months params not valid, must be integer.")
                return None
            
        start_month = time_predict + n_month
        time_observed = n_rows - start_month
        end_month = n_rows
          
        return start_month, end_month, time_predict, time_observed
    
    def create_zero_matrix(time_observed, n_cols):
        """
        Create zero matrix initializes a zero filled numpy.array matrix with adapting dimensions.
        :param n_cols: int number columns N (fund target).
        :param time_observed: time to analyze, int.
        :return: tuple with initial zero filled numpy.array objects.
        """
        if isinstance(n_cols, int):
            n_cols = n_cols
        else:
            print("number of columns params not valid, must be integer.")
            return None
        
        if isinstance(time_observed, int):
            time_observed = time_observed
        else:
            print("time observed params not valid, must be integer.")
            return None
        
        cov_matrix = np.zeros((n_cols, n_cols))
        weight_riskparity_matrix = np.zeros((time_observed, n_cols))
        
        return cov_matrix, weight_riskparity_matrix
    
    def matrix_transpose(numpy_matrix):
        """
        Matrix transpose creates an numpy.array object to its transposed shape.
        :param numpy_matrix: yield portfolio numpy.array matrix.
        :return: numpy.array object transposed.
        """
        if not isinstance(numpy_matrix, (np.ndarray)):
            print("matrix not valid, must be numpy.array object with float terms.")
            return None
            
        return numpy_matrix.T
    
    def wr_equally_weighted_matrix(time_observed, n_cols):
        """
        w & r Equally Weighted Matrix creates a zero filled numpy.array matrix
        for filling them with its ponderations.
        :param time_observed: int time window observations.
        :param n_cols: int number columns N (fund target).
        :return: numpy.array object W matrix Equally Weighted with time & cols dimensions.
        """
        if isinstance(time_observed, int):
            time_observed = time_observed
        else:
            print("time observed params not valid, must be integer.")
            return None
        
        if isinstance(n_cols, int):
            n_cols = n_cols
        else:
            print("number of columns params not valid, must be integer.")
            return None
        
        w_ew = np.zeros((time_observed, n_cols))
        r_ew  = np .zeros((time_observed, n_cols))
        
        return w_ew, r_ew
    
    def one_n_matrix(n_cols):
        """
        One/n creates a equeally weighted numpy.array matrix.
        :param n_cols: int number columns N (fund target).
        :return: numpy.array object Onen/n weights with (1 x N) dimensions.
        """
        if isinstance(n_cols, int):
            n_cols = n_cols
        else:
            print("number of columns params not valid, must be integer.")
            return None
        
        return np.full((1, n_cols), 1/n_cols)
    
    def zero_filled_vector(list_vector, time_observed):
        """
        Zero Filled Vector creates a zero filled numpy.array 1-column matrix, aka: zero-vector.
        :param list_vector: array type with string names for different type of vector.
        :param time_observed: int time window observations.
        :return: numpy.array objectZero Filled Vector with (N x 1) dimensions.
        """
        if isinstance(time_observed, int):
            time_observed = time_observed
        else:
            print("time observed params not valid, must be integer.")
            return None
        
        mapped_vec = {}
        for i in list_vector:
            mapped_vec[i] = np.zeros((time_observed, 1))
        
        return mapped_vec
    
    def return_weights_matrices(numpy_matrix, start_month, end_month, time_observed, n_cols):
        """
        One/n creates a equeally weighted numpy.array matrix.
        :param n_cols: int number columns N (fund target).
        :return: numpy.array object Onen/n weights with (1 x N) dimensions.
        """
        return_equal_weight = ModelParameters.wr_equally_weighted_matrix(time_observed=time_observed, n_cols=n_cols)[1]
        weight_matrix = ModelParameters.zero_filled_vector(list_vector=["retEW"], time_observed=time_observed)
    
        for y in range(start_month, end_month):
            return_equal_weight[y - start_month] = np.dot(numpy_matrix[y,:] , 1/n_cols)
            weight_matrix[y - start_month] = sum(return_equal_weight[y - start_month])
            
        print('Generating optimization parameters...\n')
        for y in range(start_month, end_month):
            w_EW[:] = onen
            r_ew[y - start_month] = np.dot(monthly_return[y,:] , 1/N)
            retEW[y - start_month] = sum(r_ew[y-start_month])
        %time
        print('\nDONE!')
        
        return return_equal_weight, weight_matrix

### Lets define **size** and $N$ parameters for optimization model, that will be the same as total rows and total columns; respectively.

In [516]:
size, N = ModelParameters.get_shape_matrix(numpy_matrix=monthly_return)

### Now lets define:
#### *Start month window: as the start of historical analysis for prediction.*
#### *End month window: as the end of historical analysis for prediction.*
#### *$T_1$: as the Timing 1 for prediction window (11 months).*
#### *$T_2$: as the Timing 2 for window observations (931 unique dates).*

**Note 1: Keep on-track with *size* variable, such that is the total count of historical portfolio data (943), and *$T_2$* as the total time to analyze (931), the mathematical difference between this two will be the time to predict (12 months).** 

In [517]:
start_month, end_month, T1, T2 = ModelParameters.set_time_prediction(numpy_matrix=monthly_return)

In [585]:
# Note 1 ref.
size-T2

12

### Then, lets fill with zeros *(initialize)* **covariance matrix** with dimensions $(N x N)$, and **weight Risk Parity** matrix with $(T_2 x N)$.
#### *$N$ = as Total columns from portfolio funds data (211).*
#### *$T$ = as Tistorical yeilds from portfolio funds data (931).*

In [598]:
covmatr, w_RP = ModelParameters.create_zero_matrix(time_observed=T2, n_cols=N)

In [603]:
w_RP

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

### Lets transpose **monthly returns** price difference matrix (yield to date) as *ret*. 

In [519]:
ret = ModelParameters.matrix_transpose(numpy_matrix=monthly_return)

### No lets initialize with zeros two matrices:
#### 1-. **$W$** matrix for equally weighted inputs with $(T_2 x N)$ dimensions.
#### 2-. **$R$** matrix for equally weighted inputs with $(T_2 x N)$ dinensions.

**Note 2: One matrix *w_EW* will be the matrix with equal weights with dimensions $(T_2 x N)$, .** 

In [ ]:
ret = monthly_return.T
w_EW = np.zeros((T2, N)) # no se usa
onen = np.full((1, N), 1/N) # no se usa
r_ew  = np .zeros((T2, N)) # 
r_rp = np.zeros((T2, 1))
retEW = np.zeros((T2, 1)) # 
retRP = np.zeros((T2, 1))

In [595]:
w_EW, r_ew = ModelParameters.wr_equally_weighted_matrix(time_observed=T2, n_cols=N)

In [601]:
w_EW

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

#### Lets define One/n matrix.

In [589]:
# onen = ModelParameters.one_n_matrix(n_cols=N)

#### Lets define vectors shape zero filled investment return by term as r_rp, total sum investment Return Equal Weighted as retEW, and investment Return Risk Parity.

In [545]:
lst = [
    "r_rp", 
    "retRP",
]
r_rp = ModelParameters.zero_filled_vector(lst, T2)["r_rp"]
retRP = ModelParameters.zero_filled_vector(lst, T2)["retRP"]

In [559]:
print('Generating optimization parameters...\n')
ModelParameters.return_weights_matrices(numpy_matrix=monthly_return,
                                        start_month=start_month,
                                        end_month=end_month,
                                        time_observed=T2,
                                        n_cols=N)
%time
print('\nDONE!')

Generating optimization parameters...

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs

DONE!


In [525]:
print('Generating Marginal Risk Contribution variables...\n')
for w in range(start_month, end_month):
    covmatr = np.cov(ret[:,w-T1:w])    
%time
print('\nDONE!')
print('\nvariance & covariance matrix:')
print(covmatr)

Generating Marginal Risk Contribution variables...

CPU times: user 25 µs, sys: 9 µs, total: 34 µs
Wall time: 8.34 µs

DONE!

variance & covariance matrix:
[[  1.85162577e-08   1.29703062e-08   1.96048845e-08 ...,   1.76573006e-08
    1.40867331e-08   2.12149140e-08]
 [  1.29703062e-08   9.58624981e-09   1.37871038e-08 ...,   9.63261744e-09
    1.46451919e-09   1.39949333e-08]
 [  1.96048845e-08   1.37871038e-08   2.12464350e-08 ...,   1.83229173e-08
    1.50941901e-08   2.35443789e-08]
 ..., 
 [  1.76573006e-08   9.63261744e-09   1.83229173e-08 ...,   4.45517108e-08
    8.60649809e-08   2.86959019e-08]
 [  1.40867331e-08   1.46451919e-09   1.50941901e-08 ...,   8.60649809e-08
    2.19744724e-07   4.01960130e-08]
 [  2.12149140e-08   1.39949333e-08   2.35443789e-08 ...,   2.86959019e-08
    4.01960130e-08   3.05104116e-08]]


# Generator function for Risk Contribution variables
- mrc aka: marginal risk contribution
- rc aka: risk contribution

In [ ]:
def RC(weight, covmatr):
    weight = np.array(weight)
    variance = weight.T @ covmatr @ weight
    sigma = variance ** .5
    mrc = 1/sigma * (covmatr @ weight)
    rc = weight * mrc
    rc = rc/rc.sum()
    return rc

# Generator function for RiskParity objective variables

In [ ]:
def RiskParity_objective(x):
    variance = x.T @ covmatr @ x
    sigma = variance ** .5
    mrc = 1/sigma * (covmatr @ x)
    rc = x * mrc
    a = np.reshape(rc, (len(rc),1))
    risk_diffs = a - a.T
    sum_risk_diffs_squared = np.sum(np.square(np.ravel(risk_diffs)))
    return sum_risk_diffs_squared

In [ ]:
def weight_sum_constraint(x):
    return np.sum(x) - 1.0
        
def weight_longonly(x):
    return x

# Function object for instance on Minimization scipy function

In [ ]:
def RiskParity(covmatr):
    x0 = np.repeat(1/covmatr.shape[1], covmatr.shape[1])
    constraints = ({'type': 'eq', 'fun': weight_sum_constraint},
                   {'type': 'ineq', 'fun' : weight_longonly})
    options = {'ftol' : 1e-20, 'maxiter': 999}
    result = minimize(fun = RiskParity_objective,
                      x0 = x0,
                      constraints = constraints,
                      options = options)
    return result.x

In [ ]:
print('Generating optimized return matrices...')
for w in range(start_month, end_month):
    w_RP[w - start_month] = RiskParity(covmatr)
    r_rp[w - start_month] = np.dot(monthly_return[w,:], w_RP[w - start_month,:])
    retRP[w - start_month] = sum(r_rp[w - start_month])
%time
print('\nDONE!')
print('\nw_RP matrix:')
print(w_RP)
print('\nretRP matrix:')
print(retRP)

### The x axis will be the month, the y axis will be the asset (fund), and z will be the active weight of the portfolio.

In [ ]:
mx = np.amax(w_RP)
mn = np.amin(w_RP)

fig = plt.figure()
ax = fig.gca(projection = '3d')

X = np.arange (0, T2, 1)
Y = np.arange( 0, N, 1)
X, Y = np.meshgrid(X, Y)
Z = np.transpose(w_RP)

surf = ax.plot_surface(X, Y, Z, cmap = cm.Reds_r, linewidth = 0)

ax.set_zlim(mn-.02, mx+.05)
plt.show()